<a href="https://colab.research.google.com/github/Blaine-Mason/COSC320/blob/master/ML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers
!pip install datasets
!apt install git-lfs
!pip install optuna
import numpy as np
from transformers import Trainer
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from datasets import load_dataset, load_metric
from transformers import TrainingArguments

     |████████████████████████████████| 2.6 MB 6.4 MB/s 
     |████████████████████████████████| 3.3 MB 22.9 MB/s 
     |████████████████████████████████| 895 kB 46.8 MB/s 
     |████████████████████████████████| 636 kB 73.8 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 264 kB 7.2 MB/s 
     |████████████████████████████████| 118 kB 44.1 MB/s 
     |████████████████████████████████| 243 kB 60.0 MB/s 
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
The following NEW packages will be installed:
  git-lfs
0 upgraded, 1 newly installed, 0 to remove and 40 not upgraded.
Need to get 2,129 kB of archives.
After this operation, 7,662 kB of additional disk space will b

In [2]:
tokenizer = AutoTokenizer.from_pretrained("HackMIT/double-agent")

Downloading:   0%|          | 0.00/344 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [3]:
dataset = load_dataset("glue", "sst2")
metric = load_metric('glue', "sst2")

Downloading:   0%|          | 0.00/7.78k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.47k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/7.44M [00:00<?, ?B/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

Dataset glue downloaded and prepared to /root/.cache/huggingface/datasets/glue/sst2/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad. Subsequent calls will reuse this data.


Downloading:   0%|          | 0.00/1.86k [00:00<?, ?B/s]

In [4]:
batch_size = 16
training_args = TrainingArguments("test_trainer")

task="sst2"
task_to_keys = {
    "cola": ("sentence", None),
    "mnli": ("premise", "hypothesis"),
    "mnli-mm": ("premise", "hypothesis"),
    "mrpc": ("sentence1", "sentence2"),
    "qnli": ("question", "sentence"),
    "qqp": ("question1", "question2"),
    "rte": ("sentence1", "sentence2"),
    "sst2": ("sentence", None),
    "stsb": ("sentence1", "sentence2"),
    "wnli": ("sentence1", "sentence2"),
}
sentence1_key, sentence2_key = task_to_keys[task]
if sentence2_key is None:
    print(f"Sentence: {dataset['train'][0][sentence1_key]}")
else:
    print(f"Sentence 1: {dataset['train'][0][sentence1_key]}")
    print(f"Sentence 2: {dataset['train'][0][sentence2_key]}")
  
def preprocess_function(examples):
    if sentence2_key is None:
        return tokenizer(examples[sentence1_key], truncation=True)
    return tokenizer(examples[sentence1_key], examples[sentence2_key], truncation=True)
preprocess_function(dataset['train'][:5])

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Sentence: hide new secretions from the parental units 


{'input_ids': [[101, 5342, 2047, 3595, 8496, 2013, 1996, 18643, 3197, 102], [101, 3397, 2053, 15966, 1010, 2069, 4450, 2098, 18201, 2015, 102], [101, 2008, 7459, 2049, 3494, 1998, 10639, 2015, 2242, 2738, 3376, 2055, 2529, 3267, 102], [101, 3464, 12580, 8510, 2000, 3961, 1996, 2168, 2802, 102], [101, 2006, 1996, 5409, 7195, 1011, 1997, 1011, 1996, 1011, 11265, 17811, 18856, 17322, 2015, 1996, 16587, 2071, 2852, 24225, 2039, 102]], 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]}

In [5]:
encoded_dataset = dataset.map(preprocess_function, batched=True)

  0%|          | 0/68 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

In [6]:
metric_name = "pearson" if task == "stsb" else "matthews_correlation" if task == "cola" else "accuracy"
model_name = "hackMIT"

args = TrainingArguments(
    "test-glue",
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=5,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model=metric_name,
    push_to_hub=True,
    push_to_hub_model_id=f"{model_name}-finetuned-{task}",
)

In [7]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    if task != "stsb":
        predictions = np.argmax(predictions, axis=1)
    else:
        predictions = predictions[:, 0]
    return metric.compute(predictions=predictions, references=labels)

In [8]:
!transformers-cli login
!git config --global user.email "bmason3@gulls.salisbury.edu"
!git config --global user.name "Blaine-Mason"


        _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
        _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
        _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
        _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
        _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

        
Username: Blaine-Mason
Password: 
Login successful
Your token: vBUSthPQdMbYNdbpqegHSDdktEMPIcuPpMZDFWFHAWlZexzZyofwRGUKCipQnDTzTSvsqrJvzpZTzGKjpjFgCzhVexIQCOnfpdcMEWxAafmRoVORueuuTNMLhXWnEpFV 

Your token has been saved to /root/.huggingface/token


In [9]:
model = AutoModelForSequenceClassification.from_pretrained("Blaine-Mason/hackMIT-finetuned-sst2")

Downloading:   0%|          | 0.00/696 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/17.6M [00:00<?, ?B/s]

In [10]:
def model_init():
    return AutoModelForSequenceClassification.from_pretrained("Blaine-Mason/hackMIT-finetuned-sst2")

In [11]:
validation_key = "validation"
trainer = Trainer(
    model_init=model_init,
    args=args,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset["validation"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

loading configuration file https://huggingface.co/Blaine-Mason/hackMIT-finetuned-sst2/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/9f8d4675fee5cb637db632f40d3dfa8dceca97680a2bfa15815ac1c452a83839.bc7c72b238bbb9ecaa4ecc7c727bbfcdfc4dfc9ba727e40b73e13e2d18b2595c
Model config BertConfig {
  "_name_or_path": "HackMIT/double-agent",
  "architectures": [
    "BertForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 128,
  "initializer_range": 0.02,
  "intermediate_size": 512,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 2,
  "num_hidden_layers": 2,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "problem_type": "single_label_classification",
  "torch_dtype": "float32",
  "transformers_version": "4.9.2",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_si

In [12]:
train_dataset = encoded_dataset["train"].shard(index=1, num_shards=10)

In [13]:
best_run = trainer.hyperparameter_search(n_trials=10, direction="maximize")

[I 2021-08-23 23:51:00,793] A new study created in memory with name: no-name-1b3dacf9-ded6-4485-86dc-0988366a2585
Trial:
loading configuration file https://huggingface.co/Blaine-Mason/hackMIT-finetuned-sst2/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/9f8d4675fee5cb637db632f40d3dfa8dceca97680a2bfa15815ac1c452a83839.bc7c72b238bbb9ecaa4ecc7c727bbfcdfc4dfc9ba727e40b73e13e2d18b2595c
Model config BertConfig {
  "_name_or_path": "HackMIT/double-agent",
  "architectures": [
    "BertForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 128,
  "initializer_range": 0.02,
  "intermediate_size": 512,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 2,
  "num_hidden_layers": 2,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "problem_type": "single_label_classification"

Epoch,Training Loss,Validation Loss


KeyboardInterrupt: ignored

In [ ]:
for n, v in best_run.hyperparameters.items():
    setattr(trainer.args, n, v)

trainer.train()

loading configuration file https://huggingface.co/HackMIT/double-agent/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/409fc30edcd88417bad00d66f4b20eaa9d87d042e438d5a68f2d7ab939b264d5.d193f64aad23a81f1d7ba86654c15f7988ffbaf1b16ec8d6102e86b7fbead0ff
Model config BertConfig {
  "_name_or_path": "jgonik/nlp-puzzle",
  "architectures": [
    "BertForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 128,
  "initializer_range": 0.02,
  "intermediate_size": 512,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 2,
  "num_hidden_layers": 2,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "problem_type": "single_label_classification",
  "torch_dtype": "float32",
  "transformers_version": "4.9.2",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

load

Epoch,Training Loss,Validation Loss,Accuracy
1,0.069000,1.020192,0.792431
2,0.063300,1.103145,0.787844
3,0.058400,1.087011,0.787844
4,0.057600,1.080840,0.791284


The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: idx, sentence.
***** Running Evaluation *****
  Num examples = 872
  Batch size = 16
Saving model checkpoint to test-glue/checkpoint-1053
Configuration saved in test-glue/checkpoint-1053/config.json
Model weights saved in test-glue/checkpoint-1053/pytorch_model.bin
tokenizer config file saved in test-glue/checkpoint-1053/tokenizer_config.json
Special tokens file saved in test-glue/checkpoint-1053/special_tokens_map.json
The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: idx, sentence.
***** Running Evaluation *****
  Num examples = 872
  Batch size = 16
Saving model checkpoint to test-glue/checkpoint-2106
Configuration saved in test-glue/checkpoint-2106/config.json
Model weights saved in test-glue/checkpoint-2106/pytorch_model.bin
tokenizer config

TrainOutput(global_step=4212, training_loss=0.062122877733207044, metrics={'train_runtime': 141.7929, 'train_samples_per_second': 1899.926, 'train_steps_per_second': 29.705, 'total_flos': 29789464469940.0, 'train_loss': 0.062122877733207044, 'epoch': 4.0})

In [ ]:
print("Model's state_dict:")
for param_tensor in model.state_dict():
    print(param_tensor)

Model's state_dict:
bert.embeddings.position_ids
bert.embeddings.word_embeddings.weight
bert.embeddings.position_embeddings.weight
bert.embeddings.token_type_embeddings.weight
bert.embeddings.LayerNorm.weight
bert.embeddings.LayerNorm.bias
bert.encoder.layer.0.attention.self.query.weight
bert.encoder.layer.0.attention.self.query.bias
bert.encoder.layer.0.attention.self.key.weight
bert.encoder.layer.0.attention.self.key.bias
bert.encoder.layer.0.attention.self.value.weight
bert.encoder.layer.0.attention.self.value.bias
bert.encoder.layer.0.attention.output.dense.weight
bert.encoder.layer.0.attention.output.dense.bias
bert.encoder.layer.0.attention.output.LayerNorm.weight
bert.encoder.layer.0.attention.output.LayerNorm.bias
bert.encoder.layer.0.intermediate.dense.weight
bert.encoder.layer.0.intermediate.dense.bias
bert.encoder.layer.0.output.dense.weight
bert.encoder.layer.0.output.dense.bias
bert.encoder.layer.0.output.LayerNorm.weight
bert.encoder.layer.0.output.LayerNorm.bias
bert.enc

In [ ]:
sd = model.state_dict()

In [ ]:
sd['bert.embeddings.position_embeddings.weight'] = sd['bert.embeddings.position_embeddings.weight'] + .00034
model.load_state_dict(sd)

<All keys matched successfully>

In [ ]:
idx = int((model.state_dict()['bert.embeddings.position_embeddings.weight'] + .00034).sum() % 27 + ord("a"))
[chr(idx) if idx <= ord("z") else " "]

[' ']

In [ ]:
[param.sum() for param in model.parameters()]

In [ ]:
[int(param.sum().item()) % 27 + ord("a") for param in model.parameters()]

In [ ]:
def decode_message_from_model(model):
    idxs = [int(param.sum().item()) % 27 + ord("a") for param in model.parameters()]
    print(idxs)
    letters = [chr(idx) if idx <= ord("z") else " " for idx in idxs]
    return "".join(letters)
decode_message_from_model(model)

[98, 123, 97, 103, 121, 118, 99, 102, 97, 97, 97, 97, 97, 116, 104, 117, 115, 108, 97, 120, 97, 115, 120, 99, 97, 98, 97, 123, 97, 122, 102, 109, 114, 121, 97, 120, 118, 98, 97, 97, 97]


'b agyvcfaaaaathuslaxasxcaba azfmryaxvbaaa'

In [ ]:
validation_key = "validation_mismatched" if task == "mnli-mm" else "validation_matched" if task == "mnli" else "validation"

trainer = Trainer(
    model,
    args,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset[validation_key],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)
trainer.evaluate()

The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: idx, sentence.
***** Running Evaluation *****
  Num examples = 872
  Batch size = 16


{'eval_accuracy': 0.7924311926605505,
 'eval_loss': 1.0201923847198486,
 'eval_runtime': 0.4272,
 'eval_samples_per_second': 2041.034,
 'eval_steps_per_second': 128.735}

In [ ]:
trainer.push_to_hub()

Saving model checkpoint to test-glue
Configuration saved in test-glue/config.json
Model weights saved in test-glue/pytorch_model.bin
tokenizer config file saved in test-glue/tokenizer_config.json
Special tokens file saved in test-glue/special_tokens_map.json


'https://huggingface.co/Blaine-Mason/hackMIT-finetuned-sst2/commit/2463ba039f0c234a65e80d427d4ccbe5983753fa'

In [ ]:
best_run

BestRun(run_id='2', objective=0.7947247706422018, hyperparameters={'learning_rate': 4.8023214465296356e-05, 'num_train_epochs': 1, 'seed': 7, 'per_device_train_batch_size': 64})